In [1]:
import lxml.etree as ET
import os
import numpy as np
from MultipleHash import get_hashcodes
from queue import PriorityQueue
import sys
import json
from collections import defaultdict
from statistics import mean
from scipy import stats
from math import sqrt
from numpy import std

In [2]:
data_dir = '/Users/qing/projects/TCM/datasets'
data_file = 'dblp.xml'
dtd_file = 'dblp.dtd'

In [ ]:
'''
#parse DBLP to obtain co-author dataset

dtd = ET.DTD(os.path.join(data_dir, dtd_file))  # read DTD
# iterate through nodes

records = []

for event, element in ET.iterparse(os.path.join(data_dir, data_file), dtd_validation=True):
    # print all children
    coauthors = set()
    for child in element:
        if child.tag == 'author':
            coauthors.add(child.text)
        if child.tag == 'title':
            title = child.text
        if child.tag == 'year':
            year = child.text

    if len(coauthors) < 2:
        continue
        
    records.append((title, coauthors, year))
    
print(len(records))
count = 0
output = []
f = open('coauthors.json', 'w', encoding='utf-8')
for record in records:
    json_obj = dict()
    json_obj['title'] = record[0]
    json_obj['authors'] = list(record[1])
    json_obj['year'] = record[2]
    output.append(json_obj)
    #json.dump(json_obj, f, ensure_ascii=False, indent=4)
    
output.sort(key=lambda tup: tup['year'])

json.dump(output, f, ensure_ascii=False, indent=4)
f.close()
print(len(output))

'''

In [18]:
def independent_ttest(data1, data2, alpha=0.05):

    # calculate means
    mean1, mean2 = mean(data1), mean(data2)
    # calculate standard errors
    std1, std2 = std(data1, ddof=1), std(data2, ddof=1)
    
    n1, n2 = len(data1), len(data2)
    se1, se2 = std1/sqrt(n1), std2/sqrt(n2)
    # standard error on the difference between the samples
    sed = sqrt(se1**2.0 + se2**2.0)
    # calculate the t statistic
    t_stat = (mean1 - mean2) / sed
    # degrees of freedom
    df = len(data1) + len(data2) - 2
    # calculate the critical value
    cv = stats.t.ppf(1.0 - alpha, df)
    # calculate the p-value
    p = (1.0 - stats.t.cdf(abs(t_stat), df)) * 2.0
    # return everything
    return p

In [54]:
json_reader = open('coauthors.json', 'r', encoding='utf-8')
json_objects = json.load(json_reader)
records = []
for json_obj in json_objects:
    records.append(json_obj)

In [6]:
current_year = 1930
pos = 0

triadic_closure = set()
frequency = dict()
nodes_set = set()
total_count = 0
while pos < len(records) :
    print('current year: %d' %current_year)    

    #check triadic closure
    while int(records[pos]['year']) == current_year:
        authors = records[pos]['authors']
        authors.sort()
        for i in range(len(authors)):      
            for j in range(i + 1, len(authors)):
                nodes_set.add(authors[i])
                nodes_set.add(authors[j])
                if (authors[i], authors[j]) not in frequency:
                    for node in nodes_set:
                        if((node, authors[i]) in frequency or (authors[i], node) in frequency) and ((node, authors[j]) in frequency or (authors[j], node) in frequency):
                            triadic_closure.add((node, authors[i], authors[j], current_year))
                    frequency[(authors[i], authors[j])] = dict()
                if current_year not in frequency[(authors[i], authors[j])]:
                    frequency[(authors[i], authors[j])][current_year] = 0
                frequency[(authors[i], authors[j])][current_year] += 1
                
        pos += 1
    current_year += 1

current year: 1930
current year: 1931
current year: 1932
current year: 1933
current year: 1934
current year: 1935
current year: 1936
current year: 1937
current year: 1938
current year: 1939
current year: 1940
current year: 1941
current year: 1942
current year: 1943
current year: 1944
current year: 1945
current year: 1946
current year: 1947
current year: 1948
current year: 1949
current year: 1950
current year: 1951
current year: 1952
current year: 1953
current year: 1954
current year: 1955
current year: 1956
current year: 1957
current year: 1958
current year: 1959
current year: 1960
current year: 1961
current year: 1962
current year: 1963
current year: 1964
current year: 1965
current year: 1966
current year: 1967
current year: 1968
current year: 1969
current year: 1970
current year: 1971
current year: 1972
current year: 1973
current year: 1974
current year: 1975
current year: 1976
current year: 1977
current year: 1978
current year: 1979
current year: 1980
current year: 1981
current year

KeyboardInterrupt: 

In [ ]:
current_year = 1930
pos = 0

strong_tie_set = set()
weak_tie_set = set()

connection_dict = dict()

initial_list = [0, 0, 0, 0]
threshold = 3
time_window = 4 #we look back past 4 years

statics_dict = dict()

total_count = 0
while pos < len(records):
    print('current year: %d' %current_year)    
    num_of_new_triadic_closure = 0
    
    #remove coauthor pairs that are not inactive during past 4 years
    remove_coauthor_list = []
    
    #update connection dict
    for coauthor_pair, value_list in connection_dict.items():
        connection_dict[coauthor_pair] = connection_dict[coauthor_pair][1:]
        connection_dict[coauthor_pair].append(0)
        #connection_dict[coauthor_pair] -= 1
        if sum(connection_dict[coauthor_pair]) == 0:
            remove_coauthor_list.append(coauthor_pair)
   
    for coauthor_pair in remove_coauthor_list:
        del connection_dict[coauthor_pair]
    
    #calculate triads from strong ties
    triads = set()
    strong_tie_list = list(strong_tie_set)
    for i in range(len(strong_tie_list)):        
        author_i_1, author_i_2 = strong_tie_list[i]
        for j in range(i + 1, len(strong_tie_list)):
            author_j_1, author_j_2 = strong_tie_list[j]
            if author_i_1 == author_j_2 and author_i_2 == author_j_1:
                continue
            if author_i_1 == author_j_1 and (author_i_2, author_j_2) not in strong_tie_set: #author_i_1 is the triadic node
                triads.add((author_i_1, author_i_2, author_j_2))
                triads.add((author_i_1, author_j_2, author_i_2))
            if author_i_1 == author_j_2 and (author_i_2, author_j_1) not in strong_tie_set:
                triads.add((author_i_1, author_i_2, author_j_1))    
                triads.add((author_i_1, author_j_1, author_i_2))                
            if author_i_2 == author_j_1 and (author_i_1, author_j_2) not in strong_tie_set: #author_i_2 is the triadic node
                triads.add((author_i_2, author_i_1, author_j_2))
                triads.add((author_i_2, author_j_2, author_i_1))
            if author_i_2 == author_j_2 and (author_i_1, author_j_1) not in strong_tie_set:
                triads.add((author_i_2, author_i_1, author_j_1))
                triads.add((author_i_2, author_j_1, author_i_1))
                
    #to count how many triadic_closures are formed for current year
    current_triadic_closure_set = set() 

    #check triadic closure
    while int(records[pos]['year']) == current_year:
        authors = records[pos]['authors']
        
        for i in range(len(authors)):      
            for j in range(i + 1, len(authors)):
                for (parent, child1, child2) in triads:                    
                    if authors[i] == child1 and authors[j] == child2:
                        current_triadic_closure_set.add((authors[i], authors[j]))
                        current_triadic_closure_set.add((authors[j], authors[i]))
                
                        if (authors[i], authors[j]) not in connection_dict or (authors[j], authors[i]) not in connection_dict:
                            #new coauthorship
                            #connection_dict[(authors[i], authors[j])] = initial_list
                            #connection_dict[(authors[j], authors[i])] = initial_list
                            num_of_new_triadic_closure += 1                        
                        break
                
                if (authors[i], authors[j]) not in connection_dict or (authors[j], authors[i]) not in connection_dict:
                        #new coauthorship
                        connection_dict[(authors[i], authors[j])] = initial_list
                        connection_dict[(authors[j], authors[i])] = initial_list
                        
                connection_dict[(authors[i], authors[j])][-1] = 1
                connection_dict[(authors[j], authors[i])][-1] = 1
                    
        pos += 1
    
    strong_tie_set.clear()
    weak_tie_set.clear()
    #update strong ties and weak ties
    for (author1, author2), score_list in connection_dict.items():
        score = sum(score_list)
        if score >= threshold: #strong tie
                strong_tie_set.add((author1, author2))            
                strong_tie_set.add((author2, author1))
        else: #weak tie             
                weak_tie_set.add((author1, author2))
                weak_tie_set.add((author2, author1))
                
    print("%d triads in the previous year" %(len(triads) // 2))
    print("%d newly created triadic closure" %num_of_new_triadic_closure)
    print("%d triadic closure" %(len(current_triadic_closure_set) // 2))

    statics_dict[current_year] = (len(triads) // 2, num_of_new_triadic_closure, len(current_triadic_closure_set) // 2)
    print()
    current_year += 1


In [ ]:
#print(get_hashcodes('gethashcode'))
count = 0
author_hashcodes = dict()
for authorships in coauthors_all:
    for author in authorships:
        if author not in author_hashcodes:
            hashcodes = get_hashcodes(author)
            author_hashcodes[author] = hashcodes 
            count += 1
            if count % 1000 == 0:
                print(count)
                

In [ ]:
sketch_size = 680
dimensions = [[680, 680], [680, 680], [680, 680], [680, 680], [680, 680],[620, 749],[680, 680], [680, 680],
              [749, 620], [580, 797], [797, 580], [520, 889], [889, 520],[480, 963],[963, 480], [680, 680]]
sketch_no = len(dimensions)
sketch = []
for s_no in range(sketch_no):
    sketch.append(np.zeros((dimensions[s_no][0], dimensions[s_no][1]), dtype=int))

In [ ]:
pq1 = PriorityQueue()
name_set1 = set()

pq2 = PriorityQueue()
pq22 = PriorityQueue()
name_set2 = set()

topk = 25

In [ ]:
#build sketch
count = 0
edge_count = 0
for coauthorship in coauthors_all:
    for i in range(len(coauthorship)):
        for j in range(len(coauthorship)):
            if i == j:
                continue
            edge_count += 1
            author1 = coauthorship[i]
            author2 = coauthorship[j]
            hashcodes1 = author_hashcodes[author1]
            hashcodes2 = author_hashcodes[author2]
            for s_no in range(sketch_no):
                hashcode1 = hashcodes1[s_no] % dimensions[s_no][0]
                hashcode2 = hashcodes2[s_no] % dimensions[s_no][1]
                sketch[s_no][hashcode1][hashcode2] += 1
                
            #query frequencies
            frequency = sys.maxsize
            for s_no in range(sketch_no):
                hashcode1 = hashcodes1[s_no] % dimensions[s_no][0]
                hashcode2 = hashcodes2[s_no] % dimensions[s_no][1]
                frequency = min(sketch[s_no][hashcode1][hashcode2], frequency)
                
            if author1 == 'Michael H. Böhlen':
                if author2 in name_set1:
                    pq11 = PriorityQueue()
                    while pq1.qsize() > 0:
                        fre, name = pq1.get()
                        if name != author2:
                            pq11.put((fre, name))
                        else:
                            pq11.put((frequency, author2))
                    pq1 = pq11
                else:
                    if pq1.qsize() < topk:
                        pq1.put((frequency, author2))
                        name_set1.add(author2)
                    else: 
                        frequency_in_queue, author_in_queue = pq1.get()
                        if(frequency_in_queue < frequency):
                            pq1.put((frequency, author2))
                            name_set1.remove(author_in_queue)
                            name_set1.add(author2)
                        else:
                            pq1.put((frequency_in_queue, author_in_queue))
                    
            elif author1 == 'Johann Gamper':
                if author2 in name_set2:
                    pq22 = PriorityQueue()
                    while pq2.qsize() > 0:
                        fre, name = pq2.get()
                        if name != author2:
                            pq22.put((fre, name))
                        else:
                            pq22.put((frequency, author2))
                    pq2 = pq22
                else:
                    if pq2.qsize() < topk:
                        pq2.put((frequency, author2))
                        name_set2.add(author2)
                    else: 
                        frequency_in_queue, author_in_queue = pq2.get()
                        if(frequency_in_queue < frequency):
                            pq2.put((frequency, author2))
                            name_set2.remove(author_in_queue)
                            name_set2.add(author2)
                        else:
                            pq2.put((frequency_in_queue, author_in_queue))     
                            
    count += 1
    if count % 10000 == 0:
        print(count)
print(print('edge count: %d' %edge_count))

In [ ]:
#build sketch
target1 = 'Michael H. Böhlen'
target2 = 'Johann Gamper'
name_set = set()
pq_CT = PriorityQueue()
topk = 10
count = 0
edge_count = 0

for coauthorship in coauthors_all:
    for i in range(len(coauthorship)):
        for j in range(len(coauthorship)):
            if i == j:
                continue
            edge_count += 1
            author1 = coauthorship[i]
            author2 = coauthorship[j]
            hashcodes1 = author_hashcodes[author1]
            hashcodes2 = author_hashcodes[author2]
            for s_no in range(sketch_no):
                hashcode1 = hashcodes1[s_no] % dimensions[s_no][0]
                hashcode2 = hashcodes2[s_no] % dimensions[s_no][1]
                sketch[s_no][hashcode1][hashcode2] += 1
                
            if (author1 == target1 and author2 != target2) or (author1 == target2 and author2 != target1):
                 #query frequencies
                frequency1 = sys.maxsize
                target1_hashcodes = author_hashcodes[target1]
                for s_no in range(sketch_no):
                    hashcode1 = target1_hashcodes[s_no] % dimensions[s_no][0]
                    hashcode2 = hashcodes2[s_no] % dimensions[s_no][1]
                    frequency1 = min(sketch[s_no][hashcode1][hashcode2], frequency1)
                
                frequency2 = sys.maxsize
                target2_hashcodes = author_hashcodes[target2]
                for s_no in range(sketch_no):
                    hashcode1 = target2_hashcodes[s_no] % dimensions[s_no][0]
                    hashcode2 = hashcodes2[s_no] % dimensions[s_no][1]
                    frequency2 = min(sketch[s_no][hashcode1][hashcode2], frequency2)
                    
                if frequency1 <= 0 or frequency2 <= 0:
                    continue
                
                frequency = (frequency1 * frequency2) // (frequency1 + frequency2)
                
                if author2 in name_set:
                    pq_temp = PriorityQueue()
                    while pq_CT.qsize() > 0:
                        fre, name = pq_CT.get()
                        if name != author2:
                            pq_temp.put((fre, name))
                        else:
                            pq_temp.put((frequency, author2))
                    pq_CT = pq_temp
                else:
                    if pq_CT.qsize() < topk:
                        pq_CT.put((frequency, author2))
                        name_set.add(author2)
                    else: 
                        frequency_in_queue, author_in_queue = pq_CT.get()
                        if(frequency_in_queue < frequency):
                            pq_CT.put((frequency, author2))
                            name_set.remove(author_in_queue)
                            name_set.add(author2)
                        else:
                            pq_CT.put((frequency_in_queue, author_in_queue))
                            
    count += 1
    if count % 10000 == 0:
        print(count)
print('edge count: %d' %edge_count)

In [ ]:
#print(sketch)
author1 = 'Michael H. Böhlen'
author2 = 'Johann Gamper'
sketch_indices1 = [ i % sketch_size for i in author_hashcodes[author1]]
sketch_indices2 = [ i % sketch_size for i in author_hashcodes[author2]]
topk = 5

topIndices1 = []
topIndices2 = []
for s in range(sketch_no):
    index1 = sketch_indices1[s]
    coauthors_weight1 = sketch[s][index1]
    topk_index1 = sorted(range(len(coauthors_weight1)), key=lambda i: coauthors_weight1[i])[-topk:]
    topIndices1.append(topk_index1)
    
    index2 = sketch_indices2[s]
    coauthors_weight2 = sketch[s][index2]
    topk_index2 = sorted(range(len(coauthors_weight2)), key=lambda i: coauthors_weight2[i])[-topk:]
    topIndices2.append(topk_index2)
    
    print(topk_index1)
    print(topk_index2)
    print()

In [ ]:
pq = PriorityQueue()

In [ ]:
print(pq2.queue)

In [ ]:
print(pq1.queue)

In [ ]:
print(pq_CT.queue)